<div style="width:90%;
            margin:10px;
            padding:8px;
            border:2px solid #FF0000;
            border-radius:20px;
            font-weight:bold;
            font-size:2.5em;
            text-align:center;">
JuMP benchmark : Best Linear solver, backend and solver

<div style ="width:90%;
            font-size:0.75em;
            text-align:center;">
 <i>For Goddard Rocket Problem</i></div>
</div>

<div style="width:90%;
            margin:9px;
            font-size:1em;">
            
We will compare the performance of different linear solvers, backends and solvers for the Goddard Rocket Problem using JuMP. The Goddard Rocket Problem is a classic optimal control problem where the goal is to maximize the final altitude of a rocket by controlling the thrust. 

1. [Linear solvers](#linear-solvers)
2. [Backends](#backends)
3. [Solvers](#solvers)

</div>

Importing the necessary packages and defining the problem



In [1]:
using Pkg
Pkg.activate("..")

using JuMP, OptimalControlProblems
include("./linear_solver_variant.jl")
include("./backend_variant.jl")
include("./solver_variant.jl")
include("./utils.jl")
JuMPModel = rocket(JuMPBackend();nh=50)

set_silent(JuMPModel)
set_optimizer(JuMPModel,Ipopt.Optimizer)
set_optimizer_attribute(JuMPModel,"tol",1e-8)
set_optimizer_attribute(JuMPModel,"constr_viol_tol",1e-6)
set_optimizer_attribute(JuMPModel,"max_iter",1000)
set_optimizer_attribute(JuMPModel,"sb", "yes")


  Activating project at `c:\Users\hajje\Dropbox\PC\Desktop\Stage\COTS`


## 1. Linear solver <a id='linear-solvers'></a>

In [2]:
sl_results,best_ls = linear_solver_variant(JuMPModel);

Using Ipopt Solver :
Solving With MUMPS...
******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

✅

Solving With HSLMA57...✅

Solving With HSLMA27...✅

-------------------------Results of Linear Solvers-------------------------
3×4 DataFrame
 Row │ method   diff_auto_time  total_time   objective_value 
     │ String   Float64         String       Float64         
─────┼───────────────────────────────────────────────────────
   1 │ HSLMA27      0.0079999   7.8899 ms            1.01282
   2 │ HSLMA57      0.00699997  8.20925 ms           1.01282
   3 │ MUMPS        0.0109999   13.03375 ms          1.01282


In [3]:
println("Picking the best linear solver: ")
if best_ls == "HSLMA27" 
    println("The best linear solver is HSLMA27")
    set_attribute(JuMPModel, "hsllib", HSL_jll.libhsl_path)
    set_attribute(JuMPModel, "linear_solver", "ma27")
elseif best_ls == "HSLMA57"
    println("The best linear solver is HSLMA57")
    set_attribute(JuMPModel, "hsllib", HSL_jll.libhsl_path)
    set_attribute(JuMPModel, "linear_solver", "ma57")
elseif best_ls == "MUMPS"
    println("The best linear solver is MUMPS")
    set_attribute(JuMPModel, "linear_solver", "mumps")
end

Picking the best linear solver: 
The best linear solver is HSLMA27


## 2. Backends <a id='backends'></a>

In [4]:
be_results, best_backend = backend_variant(JuMPModel);

Solving With : ExaModels...✅

Solving With : JuMPDefault (MathOptInterface.Nonlinear.SparseReverseMode())...✅

Solving With : SymbolicAD (MathOptSymbolicAD.DefaultBackend())...✅

-------------------------Results of Backends-------------------------
3×4 DataFrame
 Row │ method       diff_auto_time  total_time  objective_value 
     │ String       Float64         String      Float64         
─────┼──────────────────────────────────────────────────────────
   1 │ SymbolicAD       0.00699997  7.2113 ms           1.01282
   2 │ JuMPDefault      0.00700021  7.3951 ms           1.01282
   3 │ ExaModels        0.01        12.2675 ms          1.01282


In [5]:
println("Picking the best backend: ")
if best_backend == "SymbolicAD" 
    println("The best backend is SymbolicAD")
    set_attribute(JuMPModel,
            MOI.AutomaticDifferentiationBackend(),
            MathOptSymbolicAD.DefaultBackend(),)
elseif best_backend == "JuMPDefault"
    println("The best backend is JuMPDefault")
    set_attribute(JuMPModel,
            MOI.AutomaticDifferentiationBackend(),
            MOI.Nonlinear.SparseReverseMode(),)
elseif best_backend == "ExaModels"
    println("The best linear solver is ExaModels")
    println("ExaModels works only with ipopt and madnlp solvers")
    println("For the sake of the rest of the benchmark, we'll use the 2nd best backend $((be_results[!,1])[2])")
    second_best_backend = (be_results[!,1])[2]
    if second_best_backend == "SymbolicAD" 
        set_attribute(JuMPModel,
                MOI.AutomaticDifferentiationBackend(),
                MathOptSymbolicAD.DefaultBackend(),)
    elseif second_best_backend == "JuMPDefault"
        set_attribute(JuMPModel,
                MOI.AutomaticDifferentiationBackend(),
                MOI.Nonlinear.SparseReverseMode(),)
    end
end

Picking the best backend: 
The best backend is SymbolicAD


## 3. Solvers <a id='solvers'></a>

In [6]:
s_results,best_s = solver_variant(JuMPModel);

Solving With Ipopt...✅

Solving With MadNLP...✅

Solving With KNITRO_SQP...✅

Solving With KNITRO_IPM...✅

-------------------------Results of Solvers-------------------------
4×4 DataFrame
 Row │ method      diff_auto_time  total_time   objective_value 
     │ String      Float64         String       Float64         
─────┼──────────────────────────────────────────────────────────
   1 │ KNITRO_SQP      0.0         253.6 μs             1.01282
   2 │ KNITRO_IPM      0.0         376.6 μs             1.01282
   3 │ Ipopt           0.00999999  8.7723 ms            1.01282
   4 │ MadNLP          0.0110002   13.00505 ms          1.01282


In [7]:
println("Picking the best solver: ")
if best_s == "KNITRO_IPM" 
    println("The best solver is KNITRO_IPM")
    set_optimizer(JuMPModel,KNITRO.Optimizer);
    set_attribute(JuMPModel, "algorithm", 1);
elseif best_s == "Ipopt"
    println("The best solver is Ipopt")
    set_optimizer(JuMPModel,Ipopt.Optimizer);
elseif best_s == "MadNLP"
    println("The best solver is MadNLP")
    set_optimizer(JuMPModel,MadNLP.Optimizer);
elseif best_s == "KNITRO_SQP"
    println("The best solver is KNITRO_SQP")
    set_optimizer(JuMPModel,KNITRO.Optimizer);
    set_attribute(JuMPModel, "algorithm", 4);
end

Picking the best solver: 
The best solver is KNITRO_SQP
